In [1]:
! pip install -U langchain_community tiktoken langchain-cohere langchainhub chromadb langchain langgraph  tavily-python langchain_groq langchain_google_genai --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.2/210.2 kB 12.2 MB/s eta 0:00:0

In [2]:
import os
from google.colab import userdata
groq_api_key = userdata.get("GROQ_API_KEY")

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = userdata.get("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
langchain_api_key =userdata.get("LANGSMITH_API_KEY")

In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    groq_api_key = groq_api_key
)
print("model retrieved")

model retrieved


In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
class domainrouter(BaseModel):
    proj_domain: str = Field(
        description="answer with the domain in which the project lies in , whether in 'webapp' , 'blockchain' , 'mobile' , 'aiml'. Return only the domain name only "
    )

structured_llm_grader = llm.with_structured_output(domainrouter)
system = """You are a professional software developer who has an expertise in the web development , blockchain , aiml and mobile development role ,
    you need to classify whether the project details given by the user lies in which domain , whether it's related to website development , in this case return 'webapp' , or the project is for the blockchain domain and in this case return 'blockchain'
    whether the project is related to AIML then return with the 'aiml' domain , or else if the project contains app development then return with the 'mobile'.
    remember that this decision is very crucial dont give any kind of wrong answer. If the project has more than one domain return 'domain1+domain2' like 'webdev+aiml' """
route_domain = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "project details : {query} "),
    ]
)
domain_router = route_domain | structured_llm_grader
query = "a website for handling the user crypto wallets"
print(domain_router.invoke({"query": query}))


proj_domain='webapp+blockchain'
